In [ ]:
%load_ext autoreload
%autoreload 1
print('autoreload')

import SimpleITK as sitk
import os, sys

print('simpleitk,os,sys')

%aimport config 
sys.path.append(config.lib_dir)
%aimport utils
%aimport segment 
%aimport match 
%aimport register 
%aimport evaluate 
%aimport qc 

print('libs')

import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sbn

print('other')

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, clear_output

pd.set_option('display.max_columns', None)
# May be necessary to "Trust" the notebook in order to get the widgets to work. 

import time
import threading
#import multiprocessing
#from mantichora import mantichora

# the seaborn warnings are annoying
import warnings
warnings.filterwarnings('ignore')

# Results Visualization 

Tool for interacting with results on exahead. 


## Overview

The file `aggregated_results.csv` is formed from three different sources: 
- each core-directory's parsed names: This contains the core name and the absolute path; This can be used to track and organize the data.
- the original core segmentation stats: this will include the original file name and segmentation statistics. 
- registration results: This contains registration success metrics 

Because of this, there is substantial missingness between row observations depending on which source it came from. 

## Data Dictionary 



In [ ]:
data_dict = pd.read_csv('./workflow/libs/data_dict.csv', sep='\t')
#data_dict

In [ ]:
w = widgets.Dropdown(
    options=data_dict.name.values,
    value=data_dict.name.values[0],
    description='feature name',
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value': 
        print(data_dict[lambda x: x.name == change["new"]]['def'].item(), end='\r')

w.observe(on_change)
display(w)

# Read registration results into memory

In [ ]:
res = pd.read_csv('/mnt/e/CycIF_analysis/registration_outputs/test_core_reg/aggregated_results.csv')

# select only registered results -- keep R0 unregistered as this is the aligned reference 
res = res[(res.status == 'registered') | (res['round'] == 'R0')]

res.head()

## Available experiments 

### Choose experiment from drop down menu

In [ ]:
options = res[['slide_name', 'scene']].drop_duplicates().dropna()

opt = [f'{x.slide_name} - {x.scene}' for i,x in options.iterrows()]
_exp = widgets.Dropdown(
    options=opt,
    value=opt[0],
    description='Experiment',
    disabled=False,
)
_exp

### filter the aggregated results 

In [ ]:
SLIDE, SCENE = _exp.value.split(' - ')
res = res[(res.slide_name == SLIDE) & (res.scene == SCENE)]

## Registration results 

# Registration Results 

Why is there such clear separation between rounds? 
- something to do with lost material? I guess we would expect later rounds to register more poorly... 

Note: R1 has better metrics than R2. 

In [ ]:
# select only the DAPI results
regRes = res[res.color_channel == 'c1']
vizRes = regRes[['round',*'jacaard_coef,dice_coef,volume_similarity,false_neg_err,false_pos_err,hausdorff_dist'.split(',')]]

# basic stats
vizRes.describe()

In [ ]:
g = sbn.PairGrid(vizRes, hue='round')
g.map_upper(sbn.scatterplot)
g.map_diag(sbn.histplot)
g.map_lower(sbn.kdeplot)
g.add_legend()

## Manual thresholding 

We'll just use rational thresholds for now. Need further verification and tuning. 

In [ ]:
# for the top row, higher is better 
# for the bottom row, lower is better 

f, axes = plt.subplots(2,3, figsize=(12,6))

for name,ax in zip(vizRes.columns.values[1:], axes.flat): 
    ax.hist(vizRes[name], bins=25, density=True)
    ax.set_title(name)

# add thresholds for viz     
axes.flat[3].axvline(config.FNR_threshold, c='r')  
axes.flat[4].axvline(config.FPR_threshold, c='r')
axes.flat[5].axvline(config.hausdorff_distance_threshold, c='r')

plt.tight_layout()
plt.show()

In [ ]:
outRes = regRes[lambda x: x['dice_coef'] < 0.35] #regRes[lambda x: (x.false_pos_err > config.FPR_threshold) | (x.false_neg_err > config.FNR_threshold) | (x.hausdorff_dist > config.hausdorff_distance_threshold)]

print(f'number of outliers: {outRes.shape[0]}/{regRes.shape[0]} [{100*outRes.shape[0]/regRes.shape[0]:.1f}%]')

## inspect possible outliers

In [ ]:
# for each core 
for i, row in outRes.iterrows(): 
    #print(row) # uncomment this to see all information
    print('#'*50)
    print(row['img_name'])
    print('#'*50)
    print('jacaard:', row.jacaard_coef)
    print('dice:', row.dice_coef)
    print('volume similarity:', row.volume_similarity)
    print('FPR:', row.false_pos_err)
    print('FNR:', row.false_neg_err)
    print('hausdorff dist:', row.hausdorff_dist)
    img = sitk.ReadImage(row.registered_path)
    #utils.myshow(img)
    
    R0_dapi_path = f'{"/".join(row.registered_path.split("/")[:-1])}/unregistered_core={int(row.core)}_round=R0_color=c1.tif'
    
    R0_img = sitk.ReadImage(R0_dapi_path)
    #utils.myshow(img)
    
    f, axes = plt.subplots(1,2, figsize=(7,14))
    utils.myshow(R0_img, title='fixed', ax=axes[0])
    utils.myshow(img, title='moving', ax=axes[1])
    
    _res = evaluate.eval_registration(R0_img, img, name='for disp', plot=True)
    print('for comparison:')
    print(_res)


# Registration Visualization

Build in some interactive widgets to visualize mapping. eg scroll select for which core(1) and round(R0 + 2) to include. 

Just a convenient way to pan through and visualize images. 

In [ ]:
qc.choose_and_plot_core(config)

# Recombine images


In [ ]:
res = pd.read_csv('/home/exacloud/lustre1/NGSdev/evansna/cyclicIF/output/aggregated_results.csv')

# select only registered results -- keep R0 unregistered as this is the aligned reference 
res = res[(res.status == 'registered') | (res['round'] == 'R0')]

res.head()

# Restiching images 

### Quality Control 

This can be done using the `auto` flag, which will remove and cores that fall outside of the hardcoded thresholds (see `config.py` for details) **OR** by passing a dictionary specifying round-cores, which will be excluded from the restitched image. See comments in the cell below for details. 

In [ ]:
import json

with open('workflow/scripts/manual_QC_example.json') as data_file:    
    qc_method = json.load(data_file)
    
print(qc_method)
print(type(qc_method))

In [ ]:
## NOTE QC methods need to be further tested. ##

## change/uncomment these values to modify QC method used ###################
#qc_method = None
#qc_method = 'auto'
qc_method = {'R0':[23,3,42,35,19,79,17,22], 
             'R1':[15,12,60,54,48,38,70,62], 
             'R2':[30,46,8,24,80,9,31,44,76]}
# the dictionary here will remove cores as specified within each round. eg. 
# in this example it'll remove cores the top row of cores in R0, second row in R1... etc
# assuming we're looking at S3,Scene-1 - see tutorial.ipynb for the core id mapping (or the scene dir)
#############################################################################

### MULTI THREADED restitching ### 
threads = []
print('assigning threads...', end='\n')
for _round in np.sort(res['round'].unique()): 
    _temp = res[lambda x: (x['round'] == _round)]
    for _channel in np.sort(_temp['color_channel'].unique()): 
        print(f'\t\t\t\t {(_round, _channel)}')
        
        # parse the dictionary for manual qc. 
        if type(qc_method) == type({}): 
            if _round in qc_method.keys(): 
                _qc_method = qc_method[_round]
            else: 
                _qc_method=None
        else: 
            _qc_method = qc_method

        # see qc.py for details                                dat, _round, _channel, qc=[None, 'auto', []], output_dir='../output/S3/Scene-1', save=True, verbose=True
        t = threading.Thread(target=qc.restitch_image, args = (res, _round, _channel, _qc_method,                        '../output/S3/Scene-1/',      True,         True))
        t.daemon = True
        t.start()
        threads.append(t)
    
print('...done')
print()
print('threads are running concurrently - this may take a while ~10 minutes')
print()

# TODO: need better visualization of progress 
# prevent calls on the thread and wait for it to finish executing     
# if you call this, you can't check on the threads by querying them (next cell)
for t in threads:
    t.join()

# Visualize re-stitched images 

## **Registered** 

In [ ]:
im = sitk.ReadImage('../output/S3/Scene-1/R0_AF488.AF555.AF647.AF750_S3_2020_01_21__13471-Scene-1_c1_ORG_registered.tif')
utils.myshow(im)

In [ ]:
im = sitk.ReadImage('../output/S3/Scene-1/R1_H3K27me3.CCNB1.CCND1.Ki67_S3_2020_01_22__13485-Scene-1_c1_ORG_registered.tif')
utils.myshow(im)

In [ ]:
im = sitk.ReadImage('../output/S3/Scene-1/R2_PCNA.AR.ER.GATA3_S3_2020_01_23__13492-Scene-1_c1_ORG_registered.tif')
utils.myshow(im)

In [ ]:
qc.choose_and_viz(data_dir='../output/S3/Scene-1/')

## original / unregistered

In [ ]:
qc.choose_and_viz(data_dir='../data/')

# Deprecated Functions

In [ ]:
### SINGLE THREADED EXAMPLE ### 
print('asfadf')
dapi_images = []
for _round in np.sort(res['round'].unique()): 
    print('round:', _round)
    _temp = res[lambda x: (x['round'] == _round)]
    for _channel in np.sort(_temp['color_channel'].unique()): 
        print('\tchannel:', _channel)
        
        tic = time.time()
        joined_image = qc.restitch_image(res, _round, _channel, qc='auto', output_dir='../output/S3/', save=True)
        toc = time.time()
        print(f'\telapsed: {(toc-tic)/60:.2f}m')
        
        if _channel == 'c1': dapi_images.append(joined_image)
